# CloudClearingAPI MVP - Change Detection Demo

This notebook demonstrates satellite-based change detection to monitor development activities around Yogyakarta using Google Earth Engine and Sentinel-2 imagery.

## 🎯 Objectives
- Monitor land development changes (roads, buildings, clearing)
- Use free Sentinel-2 data via Google Earth Engine
- Implement NDVI/NDBI based change detection
- Export results for analysis and alerting

## 📋 Prerequisites
1. **Google Earth Engine Account**: Sign up at https://earthengine.google.com
2. **Authentication**: Run `earthengine authenticate` in terminal
3. **Required Libraries**: See installation commands below

## 🚀 Quick Setup
```bash
# Install required packages
pip install earthengine-api geopandas folium matplotlib
```

---

## 1. Import Required Libraries

Import necessary libraries for satellite data processing, geospatial analysis, and visualization.

In [ ]:
# Core libraries
import ee
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
import os
import warnings
warnings.filterwarnings('ignore')

# Geospatial libraries
import geopandas as gpd
from shapely.geometry import shape, Polygon, box

# Visualization libraries
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import folium
from folium import plugins
import seaborn as sns

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

print("📦 All libraries imported successfully!")
print("🌍 Ready for Earth Engine initialization...")

## 2. Configure Google Earth Engine and Authentication

Initialize Earth Engine connection. If this is your first time, you'll need to authenticate:

```bash
# Run this in terminal first (one-time setup)
earthengine authenticate
```

In [ ]:
try:
    # Initialize Earth Engine
    ee.Initialize()
    print("✅ Google Earth Engine initialized successfully!")
    
    # Test connection
    test_image = ee.Image('COPERNICUS/S2_SR/20230901T025549_20230901T030622_T48MYT')
    projection = test_image.select('B2').projection().getInfo()
    print(f"🌍 Connection test passed - Projection: {projection['crs']}")
    
except Exception as e:
    print(f"❌ Earth Engine initialization failed: {str(e)}")
    print("💡 Try running 'earthengine authenticate' in your terminal first")
    print("📖 Visit: https://developers.google.com/earth-engine/guides/python_install")

## 3. Define Area of Interest and Time Periods

Set up the monitoring area around Yogyakarta and define weekly comparison periods.

In [ ]:
# Define Area of Interest (Yogyakarta region)
# Coordinates: [West, South, East, North] in decimal degrees
AOI_COORDS = [110.25, -7.95, 110.55, -7.65]
bbox = ee.Geometry.Rectangle(AOI_COORDS)

# Calculate area for reference
area_km2 = bbox.area().divide(1000000).getInfo()

print(f"🗺️  Area of Interest: Yogyakarta Region")
print(f"📍 Coordinates: {AOI_COORDS}")
print(f"📏 Area: {area_km2:.1f} km²")

# Define time periods for comparison
# Using September 2025 as example - adjust these dates for real analysis
WEEK_A_START = '2025-09-01'  # First week
WEEK_A_END = '2025-09-07'
WEEK_B_START = '2025-09-08'  # Second week  
WEEK_B_END = '2025-09-14'

print(f"\n📅 Analysis Period:")
print(f"   Week A: {WEEK_A_START} to {WEEK_A_END}")
print(f"   Week B: {WEEK_B_START} to {WEEK_B_END}")

# Create a simple map to visualize the AOI
def create_aoi_map():
    """Create a folium map showing the area of interest"""
    center_lat = (AOI_COORDS[1] + AOI_COORDS[3]) / 2
    center_lon = (AOI_COORDS[0] + AOI_COORDS[2]) / 2
    
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=11,
        tiles='OpenStreetMap'
    )
    
    # Add AOI rectangle
    folium.Rectangle(
        bounds=[[AOI_COORDS[1], AOI_COORDS[0]], [AOI_COORDS[3], AOI_COORDS[2]]],
        color='red',
        fill=True,
        fillOpacity=0.1,
        popup='Monitoring Area'
    ).add_to(m)
    
    # Add center marker
    folium.Marker(
        [center_lat, center_lon],
        popup='Yogyakarta Monitoring Center',
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)
    
    return m

# Display the map
aoi_map = create_aoi_map()
aoi_map

## 4. Cloud Masking and Data Preprocessing

Implement cloud masking using Sentinel-2 QA60 band and apply preprocessing steps.

In [ ]:
def mask_s2_clouds(image):
    """
    Mask clouds and cirrus in Sentinel-2 image using QA60 band
    
    Args:
        image: Sentinel-2 Surface Reflectance image
        
    Returns:
        Cloud-masked image with scaled reflectance values (0-1)
    """
    qa = image.select('QA60')
    
    # Bit positions for cloud and cirrus flags
    cloud_bit_mask = 1 << 10      # Bit 10: Opaque clouds
    cirrus_bit_mask = 1 << 11     # Bit 11: Cirrus clouds
    
    # Create mask for clear pixels (both flags should be 0)
    mask = (qa.bitwiseAnd(cloud_bit_mask).eq(0)
           .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0)))
    
    # Apply mask and scale surface reflectance from [0-10000] to [0-1]
    return image.updateMask(mask).divide(10000)

def add_cloud_shadow_mask(image):
    """
    Additional cloud shadow masking using the s2cloudless algorithm
    (Optional enhancement for better cloud detection)
    """
    # Get scene classification map
    scl = image.select('SCL')
    
    # Mask cloud shadows (SCL value 3), clouds (8,9), and cirrus (10)
    clear_mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10))
    
    return image.updateMask(clear_mask)

def preprocess_sentinel2(image):
    """
    Complete preprocessing pipeline for Sentinel-2 images
    
    Args:
        image: Raw Sentinel-2 Surface Reflectance image
        
    Returns:
        Preprocessed image ready for analysis
    """
    # Apply cloud masking
    masked = mask_s2_clouds(image)
    
    # Select only the bands we need for analysis
    bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']  # Blue, Green, Red, NIR, SWIR1, SWIR2
    processed = masked.select(bands)
    
    # Add timestamp for tracking
    return processed.set('system:time_start', image.get('system:time_start'))

# Configuration parameters
MAX_CLOUD_COVER = 20  # Maximum cloud cover percentage
SCALE = 10           # Meters per pixel (Sentinel-2 resolution)

print("🛡️  Cloud masking functions defined:")
print(f"   📊 Max cloud cover: {MAX_CLOUD_COVER}%")
print(f"   📏 Processing scale: {SCALE}m per pixel")
print("   🎯 Bands: Blue, Green, Red, NIR, SWIR1, SWIR2")

## 5. Create Weekly Composite Images

Generate weekly median composites from filtered Sentinel-2 collections.

In [ ]:
def create_weekly_composite(start_date, end_date, aoi, max_cloud_cover=20):
    """
    Create weekly median composite from Sentinel-2 collection
    
    Args:
        start_date: Start date in 'YYYY-MM-DD' format
        end_date: End date in 'YYYY-MM-DD' format
        aoi: Area of interest geometry
        max_cloud_cover: Maximum cloud cover percentage
        
    Returns:
        Weekly median composite image
    """
    # Filter Sentinel-2 Surface Reflectance collection
    collection = (ee.ImageCollection('COPERNICUS/S2_SR')
                 .filterDate(start_date, end_date)
                 .filterBounds(aoi)
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover))
                 .map(preprocess_sentinel2))
    
    # Get collection info
    count = collection.size()
    
    # Create median composite and clip to AOI
    composite = collection.median().clip(aoi)
    
    # Add metadata
    composite = composite.set({
        'system:time_start': ee.Date(start_date).millis(),
        'composite_start': start_date,
        'composite_end': end_date,
        'image_count': count,
        'cloud_threshold': max_cloud_cover
    })
    
    return composite, count

# Create composites for both weeks
print("🛰️  Creating weekly composites...")

try:
    # Week A composite
    composite_a, count_a = create_weekly_composite(
        WEEK_A_START, WEEK_A_END, bbox, MAX_CLOUD_COVER
    )
    count_a_val = count_a.getInfo()
    
    # Week B composite  
    composite_b, count_b = create_weekly_composite(
        WEEK_B_START, WEEK_B_END, bbox, MAX_CLOUD_COVER
    )
    count_b_val = count_b.getInfo()
    
    print(f"✅ Week A composite: {count_a_val} images ({WEEK_A_START} to {WEEK_A_END})")
    print(f"✅ Week B composite: {count_b_val} images ({WEEK_B_START} to {WEEK_B_END})")
    
    if count_a_val == 0 or count_b_val == 0:
        print("⚠️  Warning: Some composites have no valid images!")
        print("💡 Try adjusting dates or increasing cloud cover threshold")
    
except Exception as e:
    print(f"❌ Error creating composites: {str(e)}")
    print("💡 Check your date ranges and area of interest")

## 6. Calculate Spectral Indices (NDVI, NDBI)

Compute vegetation and built-up indices for change analysis.

In [ ]:
def calculate_ndvi(image):
    """
    Calculate Normalized Difference Vegetation Index
    NDVI = (NIR - Red) / (NIR + Red)
    
    Values:
    - High NDVI (0.6-1.0): Dense vegetation
    - Medium NDVI (0.2-0.6): Sparse vegetation  
    - Low NDVI (-0.2-0.2): Bare soil, water, built areas
    - Very Low NDVI (-1.0 to -0.2): Water bodies
    """
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return ndvi.clamp(-1, 1)

def calculate_ndbi(image):
    """
    Calculate Normalized Difference Built-up Index
    NDBI = (SWIR - NIR) / (SWIR + NIR)
    
    Values:
    - High NDBI (0.3-1.0): Built-up areas, concrete, asphalt
    - Medium NDBI (0.0-0.3): Mixed built/natural areas
    - Low NDBI (-0.3-0.0): Vegetation, water
    - Very Low NDBI (-1.0 to -0.3): Dense vegetation, water bodies
    """
    ndbi = image.expression(
        '(SWIR - NIR) / (SWIR + NIR)', {
            'SWIR': image.select('B11'),  # SWIR1 band
            'NIR': image.select('B8')     # NIR band
        }).rename('NDBI')
    return ndbi.clamp(-1, 1)

def calculate_bsi(image):
    """
    Calculate Bare Soil Index
    BSI = ((SWIR + Red) - (NIR + Blue)) / ((SWIR + Red) + (NIR + Blue))
    
    Higher values indicate more exposed soil
    """
    bsi = image.expression(
        '((SWIR + RED) - (NIR + BLUE)) / ((SWIR + RED) + (NIR + BLUE))', {
            'SWIR': image.select('B11'),
            'RED': image.select('B4'),
            'NIR': image.select('B8'),
            'BLUE': image.select('B2')
        }).rename('BSI')
    return bsi.clamp(-1, 1)

def calculate_ndwi(image):
    """
    Calculate Normalized Difference Water Index
    NDWI = (Green - NIR) / (Green + NIR)
    
    Higher values indicate water presence
    """
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return ndwi.clamp(-1, 1)

# Calculate indices for both weeks
print("🔢 Calculating spectral indices...")

try:
    # Week A indices
    ndvi_a = calculate_ndvi(composite_a)
    ndbi_a = calculate_ndbi(composite_a)
    bsi_a = calculate_bsi(composite_a)
    ndwi_a = calculate_ndwi(composite_a)
    
    # Week B indices
    ndvi_b = calculate_ndvi(composite_b)
    ndbi_b = calculate_ndbi(composite_b)
    bsi_b = calculate_bsi(composite_b)
    ndwi_b = calculate_ndwi(composite_b)
    
    print("✅ NDVI: Normalized Difference Vegetation Index")
    print("✅ NDBI: Normalized Difference Built-up Index")
    print("✅ BSI: Bare Soil Index")
    print("✅ NDWI: Normalized Difference Water Index")
    
    # Get some sample statistics
    ndvi_a_stats = ndvi_a.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=bbox,
        scale=100,
        maxPixels=1e9
    ).getInfo()
    
    print(f"\n📊 Week A NDVI range: {ndvi_a_stats['NDVI_min']:.3f} to {ndvi_a_stats['NDVI_max']:.3f}")
    
except Exception as e:
    print(f"❌ Error calculating indices: {str(e)}")

## 7. Compute Change Detection Metrics

Calculate difference images and apply heuristic rules for development detection.

In [ ]:
# Change detection thresholds (adjust based on your requirements)
THRESHOLDS = {
    'ndvi_loss': -0.20,      # Significant vegetation loss
    'ndvi_gain': 0.15,       # Vegetation gain  
    'ndbi_gain': 0.15,       # Built-up area increase
    'ndbi_loss': -0.10,      # Built-up area decrease
    'bsi_gain': 0.20,        # Bare soil exposure
    'min_area_m2': 200,      # Minimum change area to flag
}

def compute_change_differences(indices_a, indices_b):
    """
    Calculate difference images between two time periods
    
    Args:
        indices_a: Dictionary of indices for time period A
        indices_b: Dictionary of indices for time period B
        
    Returns:
        Dictionary of difference images
    """
    differences = {}
    
    for index_name in indices_a.keys():
        diff = indices_b[index_name].subtract(indices_a[index_name])
        differences[f'{index_name}_diff'] = diff.rename(f'{index_name}_DIFF')
    
    return differences

def apply_change_detection_rules(differences):
    """
    Apply heuristic rules to identify different types of changes
    
    Returns:
        Dictionary of change masks for different development types
    """
    ndvi_diff = differences['NDVI_diff']
    ndbi_diff = differences['NDBI_diff']
    bsi_diff = differences['BSI_diff']
    
    # Rule 1: Development (vegetation loss + built-up gain)
    development_mask = (ndvi_diff.lt(THRESHOLDS['ndvi_loss'])
                       .And(ndbi_diff.gt(THRESHOLDS['ndbi_gain'])))
    
    # Rule 2: Land clearing (significant vegetation loss + soil exposure)
    clearing_mask = (ndvi_diff.lt(THRESHOLDS['ndvi_loss'])
                    .And(bsi_diff.gt(THRESHOLDS['bsi_gain'])))
    
    # Rule 3: Road construction (moderate NDBI increase with linear pattern)
    road_candidate_mask = (ndbi_diff.gt(0.10)
                          .And(ndvi_diff.lt(-0.10))
                          .And(ndvi_diff.gt(-0.30)))  # Not too extreme
    
    # Rule 4: Building construction (strong NDBI increase)
    building_mask = (ndbi_diff.gt(0.20)
                    .And(ndvi_diff.lt(-0.05)))
    
    # Rule 5: General vegetation loss (could be agriculture or clearing)
    veg_loss_mask = ndvi_diff.lt(THRESHOLDS['ndvi_loss'])
    
    return {
        'development': development_mask.selfMask().rename('DEVELOPMENT'),
        'clearing': clearing_mask.selfMask().rename('CLEARING'), 
        'road_candidate': road_candidate_mask.selfMask().rename('ROAD_CANDIDATE'),
        'building': building_mask.selfMask().rename('BUILDING'),
        'vegetation_loss': veg_loss_mask.selfMask().rename('VEG_LOSS')
    }

# Calculate differences between weeks
print("📊 Computing change differences...")

try:
    # Organize indices into dictionaries
    indices_a = {
        'NDVI': ndvi_a,
        'NDBI': ndbi_a, 
        'BSI': bsi_a,
        'NDWI': ndwi_a
    }
    
    indices_b = {
        'NDVI': ndvi_b,
        'NDBI': ndbi_b,
        'BSI': bsi_b, 
        'NDWI': ndwi_b
    }
    
    # Calculate differences
    differences = compute_change_differences(indices_a, indices_b)
    
    # Apply change detection rules
    change_masks = apply_change_detection_rules(differences)
    
    print("✅ Change difference images calculated")
    print("✅ Change detection rules applied")
    print(f"🎯 Using thresholds: {THRESHOLDS}")
    
    # Get basic statistics on differences
    ndvi_diff_stats = differences['NDVI_diff'].reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=bbox,
        scale=100,
        maxPixels=1e9
    ).getInfo()
    
    print(f"\n📈 NDVI Change Range: {ndvi_diff_stats['NDVI_DIFF_min']:.3f} to {ndvi_diff_stats['NDVI_DIFF_max']:.3f}")
    
except Exception as e:
    print(f"❌ Error computing changes: {str(e)}")

## 8. Apply Thresholding and Filtering

Apply morphological operations to reduce noise and false positives.

In [ ]:
def apply_morphological_filtering(image, erosion_pixels=1, dilation_pixels=2):
    """
    Apply morphological operations to reduce noise in change masks
    
    Args:
        image: Binary change mask
        erosion_pixels: Erosion kernel size
        dilation_pixels: Dilation kernel size
        
    Returns:
        Filtered change mask
    """
    # Create circular kernels
    erosion_kernel = ee.Kernel.circle(radius=erosion_pixels, units='pixels')
    dilation_kernel = ee.Kernel.circle(radius=dilation_pixels, units='pixels')
    
    # Apply morphological opening (erosion followed by dilation)
    eroded = image.focal_min(kernel=erosion_kernel, iterations=1)
    opened = eroded.focal_max(kernel=dilation_kernel, iterations=1)
    
    return opened

def combine_change_masks(change_masks, confidence_weights=None):
    """
    Combine multiple change masks into a single composite mask
    
    Args:
        change_masks: Dictionary of change masks
        confidence_weights: Optional weights for different change types
        
    Returns:
        Composite change mask with different pixel values for each change type
    """
    if confidence_weights is None:
        confidence_weights = {
            'development': 4,      # Highest confidence
            'building': 3,
            'clearing': 2,
            'road_candidate': 1,   # Needs validation
            'vegetation_loss': 1   # Could be natural
        }
    
    composite = ee.Image(0)
    
    for change_type, mask in change_masks.items():
        if change_type in confidence_weights:
            # Apply morphological filtering
            filtered_mask = apply_morphological_filtering(mask)
            
            # Add to composite with specific value
            weight = confidence_weights[change_type]
            composite = composite.add(filtered_mask.multiply(weight))
    
    return composite.selfMask()

# Apply filtering and create composite change mask
print("🔍 Applying morphological filtering...")

try:
    # Filter individual change masks
    filtered_masks = {}
    for change_type, mask in change_masks.items():
        filtered_masks[change_type] = apply_morphological_filtering(mask)
        print(f"✅ Filtered {change_type} mask")
    
    # Create composite change mask
    composite_changes = combine_change_masks(filtered_masks)
    
    # Create a high-confidence mask (development + building)
    high_confidence_changes = (filtered_masks['development']
                             .Or(filtered_masks['building']))
    
    print("✅ Morphological filtering applied")
    print("✅ Composite change mask created")
    print("✅ High-confidence change mask created")
    
    # Calculate pixel counts for each change type
    change_stats = {}
    for change_type, mask in filtered_masks.items():
        pixel_count = mask.select([0]).reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=bbox,
            scale=SCALE,
            maxPixels=1e9
        )
        change_stats[change_type] = pixel_count
    
    print(f"\n📊 Detected change pixels (10m resolution):")
    for change_type, stats in change_stats.items():
        try:
            count = list(stats.getInfo().values())[0] if stats.getInfo() else 0
            area_m2 = count * (SCALE * SCALE)
            if count > 0:
                print(f"   {change_type}: {count} pixels ({area_m2:.0f} m²)")
        except:
            print(f"   {change_type}: Could not calculate")

except Exception as e:
    print(f"❌ Error in filtering: {str(e)}")

## 9. Vectorize and Export Results

Convert change masks to vector polygons and prepare for export.

In [ ]:
def vectorize_changes(change_mask, aoi, min_area_m2=200, scale=10):
    """
    Convert change raster to vector polygons
    
    Args:
        change_mask: Binary change mask
        aoi: Area of interest geometry
        min_area_m2: Minimum polygon area in square meters
        scale: Processing scale in meters per pixel
        
    Returns:
        Feature collection of change polygons
    """
    # Convert raster to vectors
    vectors = change_mask.reduceToVectors(
        geometry=aoi,
        scale=scale,
        geometryType='polygon',
        eightConnected=False,
        labelProperty='change_value',
        maxPixels=1e9,
        tileScale=2  # Use tiling to handle large areas
    )
    
    # Add area calculation
    def add_area_and_metadata(feature):
        area = feature.geometry().area()
        perimeter = feature.geometry().perimeter()
        
        # Calculate shape metrics
        # Compactness: 4π×Area/Perimeter²  (circle = 1, line approaches 0)
        compactness = area.multiply(4 * np.pi).divide(perimeter.pow(2))
        
        return feature.set({
            'area_m2': area,
            'perimeter_m': perimeter,
            'compactness': compactness,
            'detection_date': datetime.now().strftime('%Y-%m-%d'),
            'week_a': WEEK_A_START,
            'week_b': WEEK_B_START
        })
    
    vectors_with_area = vectors.map(add_area_and_metadata)
    
    # Filter by minimum area
    filtered_vectors = vectors_with_area.filter(
        ee.Filter.gte('area_m2', min_area_m2)
    )
    
    return filtered_vectors

def export_to_drive(feature_collection, description, folder='CloudClearingAPI_Results'):
    """
    Export feature collection to Google Drive as GeoJSON
    
    Args:
        feature_collection: Earth Engine FeatureCollection
        description: Export task description
        folder: Drive folder name
        
    Returns:
        Export task object
    """
    task = ee.batch.Export.table.toDrive(
        collection=feature_collection,
        description=description,
        folder=folder,
        fileFormat='GeoJSON'
    )
    
    return task

# Vectorize high-confidence changes
print("📐 Vectorizing change polygons...")

try:
    # Vectorize high-confidence changes only
    change_polygons = vectorize_changes(
        high_confidence_changes,
        bbox,
        min_area_m2=THRESHOLDS['min_area_m2'],
        scale=SCALE
    )
    
    # Get polygon count
    polygon_count = change_polygons.size().getInfo()
    
    print(f"✅ {polygon_count} change polygons detected")
    
    if polygon_count > 0:
        # Calculate total affected area
        total_area = change_polygons.aggregate_sum('area_m2').getInfo()
        avg_area = change_polygons.aggregate_mean('area_m2').getInfo()
        
        print(f"📏 Total affected area: {total_area:.0f} m² ({total_area/10000:.2f} hectares)")
        print(f"📊 Average polygon size: {avg_area:.0f} m²")
        
        # Export to Google Drive
        export_description = f'yogya_changes_{WEEK_A_START}_to_{WEEK_B_START}'
        export_task = export_to_drive(change_polygons, export_description)
        
        # Start the export
        export_task.start()
        
        print(f"🚀 Export task started: '{export_description}'")
        print(f"📁 Check your Google Drive folder 'CloudClearingAPI_Results'")
        print(f"⏱️  Task ID: {export_task.id}")
        
        # Show task status
        print(f"📊 Task status: {export_task.status()}")
        
    else:
        print("ℹ️  No significant changes detected with current thresholds")
        print("💡 Try adjusting thresholds or time periods")
        
except Exception as e:
    print(f"❌ Error in vectorization: {str(e)}")

## 10. Visualization and Analysis

Create interactive maps and visualizations of detected changes.

In [ ]:
def create_visualization_map(composite_a, composite_b, change_mask, aoi_coords):
    """
    Create an interactive map showing before/after images and changes
    
    Args:
        composite_a: First time period composite
        composite_b: Second time period composite  
        change_mask: Detected change mask
        aoi_coords: Area of interest coordinates [west, south, east, north]
        
    Returns:
        Folium map object
    """
    # Calculate center coordinates
    center_lat = (aoi_coords[1] + aoi_coords[3]) / 2
    center_lon = (aoi_coords[0] + aoi_coords[2]) / 2
    
    # Create base map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=12,
        tiles='OpenStreetMap'
    )
    
    # Define visualization parameters for Earth Engine layers
    rgb_vis = {
        'bands': ['B4', 'B3', 'B2'],  # RGB
        'min': 0.0,
        'max': 0.3,
        'gamma': 1.2
    }
    
    ndvi_vis = {
        'bands': ['NDVI'],
        'min': -0.3,
        'max': 0.8,
        'palette': ['red', 'yellow', 'green']
    }
    
    change_vis = {
        'min': 0,
        'max': 1,
        'palette': ['red', 'orange']
    }
    
    try:
        # Add Earth Engine layers (requires folium-ee plugin)
        # This is a simplified version - in practice you'd need to use
        # the geemap library or export tiles for proper visualization
        
        # Add AOI boundary
        folium.Rectangle(
            bounds=[[aoi_coords[1], aoi_coords[0]], [aoi_coords[3], aoi_coords[2]]],
            color='red',
            weight=2,
            fill=False,
            popup='Analysis Area'
        ).add_to(m)
        
        # Add layer control
        folium.LayerControl().add_to(m)
        
        # Add a marker for reference
        folium.Marker(
            [center_lat, center_lon],
            popup=f'Yogyakarta Change Detection<br>Period: {WEEK_A_START} to {WEEK_B_START}',
            icon=folium.Icon(color='red', icon='search')
        ).add_to(m)
        
    except Exception as e:
        print(f"Warning: Could not add Earth Engine layers: {e}")
    
    return m

def plot_change_statistics():
    """
    Create plots showing change detection statistics
    """
    try:
        # Sample data for demonstration (replace with actual statistics)
        change_types = ['Development', 'Building', 'Clearing', 'Road Candidate', 'Vegetation Loss']
        areas = [1500, 800, 2200, 600, 3200]  # Sample areas in m²
        
        # Create subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Bar plot of change areas
        colors = ['#ff4444', '#ff8844', '#ffaa44', '#44aa88', '#44aaff']
        bars = ax1.bar(change_types, areas, color=colors, alpha=0.7)
        ax1.set_title('Detected Changes by Type', fontsize=14, fontweight='bold')
        ax1.set_ylabel('Area (m²)')
        ax1.tick_params(axis='x', rotation=45)
        
        # Add value labels on bars
        for bar, area in zip(bars, areas):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + 50,
                    f'{area}m²', ha='center', va='bottom')
        
        # Pie chart of change distribution
        ax2.pie(areas, labels=change_types, colors=colors, autopct='%1.1f%%', startangle=90)
        ax2.set_title('Change Distribution', fontsize=14, fontweight='bold')
        
        plt.tight_layout()
        plt.show()
        
        # Summary statistics
        total_area = sum(areas)
        print(f"\n📊 Change Detection Summary:")
        print(f"   Total affected area: {total_area:,} m² ({total_area/10000:.2f} hectares)")
        print(f"   Number of change types: {len([a for a in areas if a > 0])}")
        print(f"   Largest change type: {change_types[areas.index(max(areas))]} ({max(areas):,} m²)")
        
    except Exception as e:
        print(f"Error creating plots: {e}")

def create_change_timeline():
    """
    Create a timeline visualization of the analysis
    """
    try:
        # Create timeline
        fig, ax = plt.subplots(figsize=(12, 4))
        
        # Timeline points
        dates = [WEEK_A_START, WEEK_B_START]
        labels = ['Week A\n(Baseline)', 'Week B\n(Comparison)']
        
        # Plot timeline
        ax.plot(dates, [1, 1], 'ro-', linewidth=3, markersize=10)
        
        # Add labels
        for i, (date, label) in enumerate(zip(dates, labels)):
            ax.annotate(label, (date, 1), xytext=(0, 20), 
                       textcoords='offset points', ha='center',
                       fontsize=12, fontweight='bold')
        
        ax.set_ylim(0.5, 1.5)
        ax.set_title('Analysis Timeline', fontsize=16, fontweight='bold')
        ax.set_xlabel('Date')
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis='y', left=False, labelleft=False)
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error creating timeline: {e}")

# Create visualizations
print("🗺️  Creating visualizations...")

try:
    # Create main visualization map
    viz_map = create_visualization_map(composite_a, composite_b, high_confidence_changes, AOI_COORDS)
    
    print("✅ Interactive map created")
    
    # Display the map
    display(viz_map)
    
    # Create statistical plots
    plot_change_statistics()
    
    # Create timeline
    create_change_timeline()
    
    print("✅ All visualizations created successfully!")
    
except Exception as e:
    print(f"❌ Error creating visualizations: {str(e)}")
    
# Display summary
print(f"\n🎉 Analysis Complete!")
print(f"📊 Area analyzed: {area_km2:.1f} km² around Yogyakarta")
print(f"📅 Time period: {WEEK_A_START} to {WEEK_B_START}")
print(f"🛰️  Data source: Sentinel-2 (10m resolution)")
print(f"⚙️  Algorithm: NDVI/NDBI difference analysis")
print(f"📁 Results exported to Google Drive")